# Flights vs. Weather Project

## Setting up environment

### Copy .env and sql_functions from previous notebooks

In [1]:
# ! cp /Users/danielschuster/neuefische/Git/forked-da-external-data-sourcing/.env .
# ! cp /Users/danielschuster/neuefische/Git/forked-da-external-data-sourcing/sql_functions.py .

## TASK 1: Historical Weather Event --> Hurricane Katrina, August 23, 2005

### Connect to Meteostat API

In [7]:
weather_katrina_df = pd.json_normalize(weather_katrina, 
                                       sep="_", 
                                       record_path="data", 
                                    #    meta=[["meta", "generated"], 
                                    #          ["meta", "stations"]],
                                       record_prefix="data_",
                                       errors='ignore')
weather_katrina_df.head()

NameError: name 'pd' is not defined

### OTHER AIRPORTS FOR DATAFRAMES

* New Orleans, Ln, MSY --> done
* Baton Rouge, La, BTR --> Daniel
* Biloxi, Miss, GPT --> Daniel
* Mobile, Ala, MOB --> Juls
* Pensacola, Fla, PNS --> Juls
* Eglin Air Force Base, Fla, VPS --> Juls

# Getting Weather Data near MSY, BTR and GPT airports during Katrina using get_weather_data.get_katrina_weather_data()

### BTR (Baton Rouge, La)

In [1]:
from get_weather_data import get_katrina_weather_data as get_kat_data

btr_data = get_kat_data('30.53236', '-91.151461')
btr_data.head()

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2005-08-01,27.9,24.4,33.3,11.7,None,NaN,4.3,None,1016.4,None
1,2005-08-02,28.1,24.0,33.4,0.0,None,NaN,3.8,None,1015.5,None
2,2005-08-03,27.3,25.1,31.2,0.0,None,NaN,5.0,None,1014.8,None
3,2005-08-04,26.1,23.4,32.3,10.9,None,NaN,6.3,None,1016.4,None
4,2005-08-05,27.0,22.2,31.7,0.0,None,NaN,10.1,None,1017.6,None


### GPT (Biloxi, Miss)

In [2]:
gpt_data = get_kat_data('30.413366', '-89.072955')
gpt_data.head()

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2005-08-01,25.8,23.3,31.1,22.9,0.0,NaN,4.0,None,1016.1,None
1,2005-08-02,NaN,23.3,30.6,0.8,0.0,NaN,5.4,None,NaN,None
2,2005-08-03,25.1,21.7,31.1,10.4,0.0,NaN,9.0,None,NaN,None
3,2005-08-04,24.3,22.2,30.6,25.7,0.0,NaN,7.2,None,NaN,None
4,2005-08-05,26.2,21.7,31.7,0.0,0.0,NaN,10.1,None,1016.9,None


### MSY (New Orleans, Ln)

In [3]:
msy_data = get_kat_data('29.9893', '-90.2548')
msy_data.head()

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2005-08-01,27.9,24.4,33.3,11.7,0.0,NaN,4.3,None,1016.4,None
1,2005-08-02,27.0,23.9,31.1,9.9,0.0,NaN,5.8,None,1015.9,None
2,2005-08-03,27.9,24.4,31.1,1.0,0.0,NaN,7.6,None,1015.1,None
3,2005-08-04,26.6,23.9,30.0,10.9,0.0,NaN,8.3,None,1016.7,None
4,2005-08-05,27.0,22.2,31.7,0.0,0.0,NaN,10.1,None,1017.6,None


# Basic Data Cleaning

Dropping 'dwpt', 'rhum', 'wdir', 'snow', 'tsun', 'wpgt', 'coco' and 'prcp' either because information is not valuable or all values are "None"

In [4]:
from get_weather_data import clean_daily_data

msy_clean = clean_daily_data(msy_data, 'MSY')
msy_clean.head()

,date,avg_temp_celsius,wind_speed_kph,air_pressure_hPa,airport_code
0,2005-08-01,27.9,4.3,1016.4,MSY
1,2005-08-02,27.0,5.8,1015.9,MSY
2,2005-08-03,27.9,7.6,1015.1,MSY
3,2005-08-04,26.6,8.3,1016.7,MSY
4,2005-08-05,27.0,10.1,1017.6,MSY


In [5]:
btr_clean = clean_daily_data(btr_data, 'BTR')
btr_clean.head()

,date,avg_temp_celsius,wind_speed_kph,air_pressure_hPa,airport_code
0,2005-08-01,27.9,4.3,1016.4,BTR
1,2005-08-02,28.1,3.8,1015.5,BTR
2,2005-08-03,27.3,5.0,1014.8,BTR
3,2005-08-04,26.1,6.3,1016.4,BTR
4,2005-08-05,27.0,10.1,1017.6,BTR


In [6]:
gpt_clean = clean_daily_data(gpt_data, 'GPT')
gpt_clean.head()

,date,avg_temp_celsius,wind_speed_kph,air_pressure_hPa,airport_code
0,2005-08-01,25.8,4.0,1016.1,GPT
1,2005-08-02,NaN,5.4,NaN,GPT
2,2005-08-03,25.1,9.0,NaN,GPT
3,2005-08-04,24.3,7.2,NaN,GPT
4,2005-08-05,26.2,10.1,1016.9,GPT


In [7]:
from sql_functions import get_engine
from sql_functions import push_to_database
import psycopg2

schema = 'cgn_analytics_22_3' # UPDATE 'TABLE_SCHEMA' based on schema used in class 
engine = get_engine()
table_name = 'Group_1_Weather_MSY'

push_to_database(msy_clean, table_name, engine=engine, schema=schema)

UnboundLocalError: local variable 'engine' referenced before assignment